In [ ]:
!py -3 -m pip install --upgrade Jinja2==3.0.3

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import sqlalchemy
import csv
import os
from jinjasql import JinjaSql

In [ ]:
user_name = 'KHU9683'
pword = 'Mer82@dme'
host_name='EDWPROD.DW.MEDCITY.NET'

In [ ]:
td_engine = sqlalchemy.create_engine('teradatasql://' + host_name + '/?user=' + user_name + '&password=' + pword + '&logmech=LDAP')

In [ ]:
def read_contents_file(file_path, encoding_scheme='cp1252'):
    inputfile = open(file_path, 'r', encoding=encoding_scheme)
    return inputfile.read()

In [ ]:
#Write file to local directory
def write_file_local(path,file_data):
    
    with open(path, 'w') as file:
        file_string = '\n'.join(file_data)
        file.write(file_string)

In [ ]:
#path of file with table name and database name in csv file
ddlgen_template_path = "InputFiles\\ddl_gen.sql"
input_path = "OutputFiles\\DDL\\"
input_file_extension = "csv"
output_path_folder = "OutputFiles\\DDL\\"
output_file_extension = "sql"
schema_prefix = "edwfs"

In [ ]:
#get the DDLs for the tables from teradata 
def gen_bq_tables_ddl():
        dt1 = datetime.now()
        # database_name = "'edwpi'"
        # table_name = "'bi_department_daily_activity','bi_department_pp_activity'"
        DDL_GEN_TEMPLATE = read_contents_file(ddlgen_template_path)
        
        delimiter = ','
        quote_char = '"'
        input_path_folder = f"{input_path}\{schema_prefix}.{input_file_extension}"
        df = pd.read_csv(input_path_folder, index_col=None)
        df = df[(df.Object_Type != "V")]
        df_database = df.Database.unique()

        run_date = (datetime.now()+timedelta(hours=-6)).strftime('%Y%m%d')

        for database in df_database:
            try:
                database_name = database.strip().lower()
                if not os.path.exists(f"{output_path_folder}{database_name}"):
                    os.makedirs(f"{output_path_folder}{database_name}")      

                objects = df[(df.Database == database_name)]
                database_name_fmt = "'{}'".format(database_name)
                table_list = ','.join("'{}'".format(obj) for obj in objects['Object'].to_list())

                query = DDL_GEN_TEMPLATE.format(database_name_fmt, table_list)
                # print (query)
                tableddl_df = pd.read_sql(query, td_engine)

                for index, tableddl_rec in tableddl_df.iterrows():
                    object_name = str(tableddl_rec['table_name'])
                    table_output_path = f"{output_path_folder}{database_name}\{object_name}.sql"
                    table_definition = [str(tableddl_rec['ddl'])]
                    write_file_local(table_output_path, table_definition)

            except Exception as e1:
                print(e1)
                print("Database : {}, Table : {}".format(database_name,table_list))
                pass
        
        dt2 = datetime.now()
        print(dt2-dt1)
        print("Tables for Writing DDL : ", len(df))

In [ ]:
print("Begin of Processing")

gen_bq_tables_ddl()

print("End of Processing")